In [4]:
# Setup: install dependencies for 3D rendering
import sys, subprocess, pkgutil

def ensure(pkg):
    if pkgutil.find_loader(pkg) is None:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', pkg])

for p in ['plotly', 'kaleido']:
    ensure(p)

import plotly
print('plotly', plotly.__version__)
print('Ready.')


C:\Users\pc\AppData\Local\Temp\ipykernel_14508\1132745681.py:5: DeprecationWarning: 'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead
  if pkgutil.find_loader(pkg) is None:


plotly 6.4.0
Ready.


In [5]:
# Cube 3D Visualizations (per fact)
# This notebook renders conceptual OLAP cubes (3 axes) for each fact table
# and saves PNGs under assets/spritesheets_datacube/cubes/.

from pathlib import Path

import plotly.graph_objects as go

OUTPUT_DIR = Path('../assets/spritesheets_datacube/cubes')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print('Output dir:', OUTPUT_DIR.resolve())


Output dir: C:\Users\pc\OneDrive\Desktop\INTELI\2025-2B-T14-SI08-G01\assets\spritesheets_datacube\cubes


In [ ]:
def make_wire_cube(edge=1.0, color='rgba(30,30,30,0.6)'):
    # 12 edges of a cube
    e = edge
    lines = [
        ((0, 0, 0), (e, 0, 0)), ((0, e, 0), (e, e, 0)), ((0, 0, e), (e, 0, e)), ((0, e, e), (e, e, e)),
        ((0, 0, 0), (0, e, 0)), ((e, 0, 0), (e, e, 0)), ((0, 0, e), (0, e, e)), ((e, 0, e), (e, e, e)),
        ((0, 0, 0), (0, 0, e)), ((e, 0, 0), (e, 0, e)), ((0, e, 0), (0, e, e)), ((e, e, 0), (e, e, e)),
    ]
    xs, ys, zs = [], [], []
    for (x1, y1, z1), (x2, y2, z2) in lines:
        xs += [x1, x2, None]
        ys += [y1, y2, None]
        zs += [z1, z2, None]
    return go.Scatter3d(x=xs, y=ys, z=zs, mode='lines', line=dict(color=color, width=5), showlegend=False)


def render_cube_png(title: str, x_label: str, y_label: str, z_label: str, measures_label: str, out_png: Path):
    fig = go.Figure()
    fig.add_trace(make_wire_cube())

    # Axis labels
    fig.add_trace(go.Scatter3d(x=[1.05], y=[0], z=[0], mode='text', text=[x_label], textposition='middle right'))
    fig.add_trace(go.Scatter3d(x=[0], y=[1.05], z=[0], mode='text', text=[y_label], textposition='middle right'))
    fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[1.05], mode='text', text=[z_label], textposition='middle right'))

    # Measures at cube center
    fig.add_trace(go.Scatter3d(x=[0.5], y=[0.5], z=[0.5], mode='markers+text',
                               text=[measures_label], textposition='top center',
                               marker=dict(size=8, color='#ffb84d')))

    fig.update_layout(
        title=title,
        scene_camera=dict(eye=dict(x=1.7, y=1.1, z=1.2)),
        scene=dict(
            xaxis=dict(title='', showbackground=False, showticklabels=False, zeroline=False),
            yaxis=dict(title='', showbackground=False, showticklabels=False, zeroline=False),
            zaxis=dict(title='', showbackground=False, showticklabels=False, zeroline=False),
            aspectmode='cube',
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        showlegend=False,
    )
    try:
        fig.write_image(str(out_png), width=900, height=900, scale=2)
        print('Saved:', out_png)
    except ValueError as e:
        print('Warning: kaleido not installed, skipping PNG save. Install with: pip install kaleido')
    return fig


In [28]:
# Optional: voxel-style OLAP cube with per-cell numbers (matplotlib)
import sys, subprocess, pkgutil
if pkgutil.find_loader('matplotlib') is None:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', 'matplotlib'])

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm


def render_voxel_cube(values3d: np.ndarray,
                      x_labels, y_labels, z_labels,
                      title: str, out_png: Path,
                      cmap_name: str = 'viridis', alpha: float = 0.75,
                      value_fmt: str = '.1f',
                      axis_titles: tuple = ('X', 'Y', 'Z'),
                      colorbar_label: str = 'Measure',
                      fill_all: bool = True,
                      view: str = 'front'):
    """Render a 3D voxel cube with numeric values printed in each cell.
    values3d shape: (nx, ny, nz). If fill_all=True, todos os voxels são mostrados (zeros ficam com cor mínima).
    """
    nx, ny, nz = values3d.shape
    filled = np.ones_like(values3d, dtype=bool) if fill_all else (values3d != 0)

    # Normalize colors by value
    vmin = float(values3d.min()) if fill_all else float(values3d[filled].min() if filled.any() else 0)
    vmax = float(values3d.max()) if fill_all else float(values3d.max() if filled.any() else 1)
    if vmax <= vmin:
        vmax = vmin + 1.0
    norm = plt.Normalize(vmin=vmin, vmax=vmax)
    cmap = cm.get_cmap(cmap_name)

    facecolors = np.empty(filled.shape + (4,), dtype=float)
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                if filled[i, j, k]:
                    rgba = list(cmap(norm(values3d[i, j, k])))
                    rgba[3] = alpha
                    facecolors[i, j, k] = rgba
                else:
                    facecolors[i, j, k] = (0, 0, 0, 0)

    fig = plt.figure(figsize=(9, 8), dpi=200)
    ax = fig.add_subplot(111, projection='3d')
    ax.voxels(filled, facecolors=facecolors, edgecolor='gray', linewidth=0.4)

    # Text labels at centers
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                if filled[i, j, k]:
                    ax.text(i + 0.5, j + 0.5, k + 0.5, f"{values3d[i,j,k]:{value_fmt}}",
                            ha='center', va='center', color='black', fontsize=7)

    # Axis labels and ticks
    ax.set_title(title, fontsize=12)
    ax.set_xlabel(axis_titles[0], labelpad=10)
    ax.set_ylabel(axis_titles[1], labelpad=10)
    ax.set_zlabel(axis_titles[2], labelpad=10)
    ax.set_xticks(np.arange(nx) + 0.5)
    ax.set_yticks(np.arange(ny) + 0.5)
    ax.set_zticks(np.arange(nz) + 0.5)
    ax.set_xticklabels(x_labels, rotation=0, fontsize=8)
    ax.set_yticklabels(y_labels, fontsize=8)
    ax.set_zticklabels(z_labels, fontsize=8)

    # View presets
    if view == 'front':
        ax.view_init(elev=18, azim=-15)
    elif view == 'isometric':
        ax.view_init(elev=22, azim=-35)
    elif view == 'top':
        ax.view_init(elev=80, azim=-35)
    else:
        ax.view_init(elev=22, azim=-35)

    # Colorbar
    from matplotlib.cm import ScalarMappable
    mappable = ScalarMappable(norm=norm, cmap=cmap)
    mappable.set_array(values3d)
    cbar = fig.colorbar(mappable, ax=ax, shrink=0.6, pad=0.05)
    cbar.set_label(colorbar_label)

    fig.subplots_adjust(left=0.02, right=0.96, bottom=0.06, top=0.9)
    fig.savefig(out_png, bbox_inches='tight')
    plt.close(fig)
    print('Saved voxel cube:', out_png)


C:\Users\pc\AppData\Local\Temp\ipykernel_14508\2936733467.py:3: DeprecationWarning:

'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead



In [29]:
# Demo with synthetic values (4 x 4 x 4)
np.random.seed(42)
values = np.random.randint(0, 100, size=(4,4,4)).astype(float)
# zero-out some cells to create gaps
values[values < 20] = 0

x_labels = ['Company A','Company B','Company C','Company D']
y_labels = ['Jan','Feb','Mar','Apr']
z_labels = ['Cust 1','Cust 2','Cust 3','Cust 4']

render_voxel_cube(values, x_labels, y_labels, z_labels,
                   title='Voxel Cube — demo (values shown per cell)',
                   out_png=OUTPUT_DIR / 'demo_voxel_cube.png')


C:\Users\pc\AppData\Local\Temp\ipykernel_14508\2936733467.py:32: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.



Saved voxel cube: ..\assets\spritesheets_datacube\cubes\demo_voxel_cube.png


In [30]:
# Helper: pivot a DataFrame (x,y,z,value) into a dense 3D array
import pandas as pd

def pivot_to_3d(df: pd.DataFrame, x: str, y: str, z: str, value: str):
    xs = sorted(df[x].unique())
    ys = sorted(df[y].unique())
    zs = sorted(df[z].unique())
    nx, ny, nz = len(xs), len(ys), len(zs)
    arr = np.zeros((nx, ny, nz), dtype=float)
    index = {(row[x], row[y], row[z]): row[value] for _, row in df.iterrows()}
    for i, xv in enumerate(xs):
        for j, yv in enumerate(ys):
            for k, zv in enumerate(zs):
                arr[i, j, k] = float(index.get((xv, yv, zv), 0))
    return arr, xs, ys, zs


In [31]:
# Compact demo (3x4x3) with clear labels and colorbar
np.random.seed(7)
arr = np.random.randint(0, 100, size=(3,4,3)).astype(float)
arr[arr < 25] = 0
x = ['Company A','Company B','Company C']
y = ['Jan','Feb','Mar','Apr']
z = ['Cust 1','Cust 2','Cust 3']
render_voxel_cube(
    arr, x, y, z,
    title='fact_sales — voxel cube (demo values)',
    out_png=OUTPUT_DIR / 'fact_sales_voxel_demo.png',
    axis_titles=('Company','Month','Customer'),
    colorbar_label='valor_total',
    fill_all=True,
    view='front'
)


C:\Users\pc\AppData\Local\Temp\ipykernel_14508\2936733467.py:32: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.



Saved voxel cube: ..\assets\spritesheets_datacube\cubes\fact_sales_voxel_demo.png


In [27]:
cubes = [
    ("fact_sales — 3D view", "dim_date", "dim_company", "dim_customer", "valor_total / icms / ipi", OUTPUT_DIR / "fact_sales_cube.png"),
    ("fact_purchases — 3D view", "dim_date", "dim_company", "dim_supplier", "valor_total / icms / ipi", OUTPUT_DIR / "fact_purchases_cube.png"),
    ("fact_services — 3D view", "dim_date", "dim_company", "dim_customer", "valor_servico", OUTPUT_DIR / "fact_services_cube.png"),
    ("fact_inventory_movements — 3D view", "dim_document", "dim_company", "dim_product", "qtd / v_unit / v_total", OUTPUT_DIR / "fact_inventory_movements_cube.png"),
    ("fact_accounting_entries — 3D view", "dim_date", "dim_company", "dim_cost_center", "valor (deb/cred)", OUTPUT_DIR / "fact_accounting_entries_cube.png"),
    ("fact_payroll — 3D view", "dim_date", "dim_company", "dim_payroll_event", "valor", OUTPUT_DIR / "fact_payroll_cube.png"),
]

figs = []
for title, xl, yl, zl, ml, out_file in cubes:
    fig = render_cube_png(title, xl, yl, zl, ml, out_file)
    figs.append(fig)

print('Done. If PNGs did not save, run: pip install kaleido')


Saved: ..\assets\spritesheets_datacube\cubes\fact_sales_cube.png
Saved: ..\assets\spritesheets_datacube\cubes\fact_purchases_cube.png
Saved: ..\assets\spritesheets_datacube\cubes\fact_services_cube.png
Saved: ..\assets\spritesheets_datacube\cubes\fact_inventory_movements_cube.png
Saved: ..\assets\spritesheets_datacube\cubes\fact_accounting_entries_cube.png
Saved: ..\assets\spritesheets_datacube\cubes\fact_payroll_cube.png
Done. If PNGs did not save, run: pip install kaleido
